<a href="https://colab.research.google.com/github/Konstantinosil/Tsioutsios-Linardatos-Persistence-in-the-Output-Gap-for-the-G7/blob/main/Persistence_in_the_Output_Gap_for_the_G7_Tsioutsios_%26_Linardatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import warnings, numpy as np, pandas as pd
from scipy import optimize, linalg
import statsmodels.api as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

DATA_PATH = "/content/total dataset.xlsx"
G7_SHEETS = None
MIN_OBS = 20
BOOT_B = 299
BOOT_L = 8
RNG_SEED = 123

BASELINE_POLICY = dict(sample_policy="overlap", use_diff=False)
ROBUSTNESS_POLICY = dict(sample_policy="balanced", use_diff=True)

PAIR_POSITIONS = [
    ("gdp",  0, 1),
    ("cpi",  2, 3),
    ("rate", 4, 5),
    ("unemp",6, 7),
    ("ip",   8, 9)
]

def _to_month_end(dt_ser):
    dt = pd.to_datetime(dt_ser, errors="coerce")
    return pd.DatetimeIndex(dt).to_period("M").to_timestamp(how="end")

def zscore(s: pd.Series):
    s = pd.to_numeric(s, errors="coerce").astype(float)
    m, sd = s.mean(), s.std()
    return (s - m)/sd if (sd is not None and sd > 0) else s*0.0

def half_life_from_d(d):
    if d is None or (isinstance(d, float) and np.isnan(d)):
        return 1.0
    if d <= 0:
        return 1.0

    return float((1.0 / max(2e-12, 2.0*d)) * 12.0)

def format_halflife_months(hl, soft_cap=10000):
    if hl is None or (isinstance(hl, float) and np.isnan(hl)) or hl <= 0:
        return "1 month"
    if hl > soft_cap:
        return f">{soft_cap:,} months"
    return f"{int(round(hl)):,} months"

def parse_sheet_by_position(df_raw: pd.DataFrame):
    series = {}
    ncols = df_raw.shape[1]
    for key, dcol, vcol in PAIR_POSITIONS:
        if dcol < ncols and vcol < ncols:
            dt = _to_month_end(df_raw.iloc[:, dcol])
            vals = pd.to_numeric(df_raw.iloc[:, vcol], errors="coerce")
            s = pd.Series(vals.values, index=dt).groupby(level=0).last().sort_index()
            if s.notna().sum() >= 3:
                series[key] = s
    return series

def build_monthly_panel(pairs):
    all_idx = None
    for s in pairs.values():
        all_idx = s.index if all_idx is None else all_idx.union(s.index)
    all_idx = pd.DatetimeIndex(all_idx).sort_values()
    idx_m = pd.date_range(all_idx.min(), all_idx.max(), freq="M")
    df = pd.DataFrame(index=idx_m)
    for k, s in pairs.items():
        df[k] = s.reindex(idx_m)
    return df

def quick_data_audit(pairs):
    print("---- DATA AUDIT (raw pairs) ----")
    for k, s in pairs.items():
        ss = s.dropna()
        if len(ss)==0:
            print(f"{k:>6}: (empty)")
        else:
            print(f"{k:>6}: n={len(ss)}, start={ss.index.min().date()}, end={ss.index.max().date()}")

def quick_panel_audit(dfm):
    print("---- PANEL AUDIT (union monthly) ----")
    for c in dfm.columns:
        nn = int(dfm[c].notna().sum())
        if nn>0:
            first = dfm[c].dropna().index.min().date()
            last  = dfm[c].dropna().index.max().date()
            print(f"{c:>10}: n={nn}, start={first}, end={last}")

def transforms(dfm, use_diff=False):
    out = pd.DataFrame(index=dfm.index).copy()

    if "gdp" in dfm.columns:
        g = dfm["gdp"].where(dfm["gdp"] > 0)
        out["gdp_log"] = np.log(g)

    if "cpi" in dfm.columns:
        p = dfm["cpi"].where(dfm["cpi"] > 0)
        pl = np.log(p)
        out["infl_yoy"] = 100*(pl - pl.shift(12))
        if "gdp_log" in out:
            out["infl_q_ann"] = ((pl - pl.shift(3))*(400/3.0)).where(out["gdp_log"].notna())

    if "ip" in dfm.columns:
        ip = dfm["ip"].where(dfm["ip"] > 0)
        out["ip_log"] = np.log(ip)

    if "unemp" in dfm.columns:
        out["unemp"] = dfm["unemp"]

    if "rate" in dfm.columns:
        out["rate"] = dfm["rate"]

    if use_diff:
        for c in ["infl_yoy", "infl_q_ann", "ip_log", "unemp", "rate"]:
            if c in out.columns:
                out[c] = out[c].diff()

    out_std = out.apply(zscore)
    return out, out_std

def fit_uc_quarterly(country, can):
    if "gdp_log" not in can or can["gdp_log"].dropna().empty:
        print(f"[{country}] No GDP to fit UC.")
        return None, None

    y = can["gdp_log"].dropna()
    mod = UnobservedComponents(
        endog=y,
        level='local linear trend',
        cycle=True,
        stochastic_level=True,
        stochastic_trend=True,
        stochastic_cycle=True,
        damped_cycle=False
    )
    try:
        res = mod.fit(disp=False, maxiter=500)
    except Exception as e:
        print(f"[{country}] UC failed: {e}")
        return None, None

    try:
        gap_q = res.cycle.smoothed.rename(f"{country}_gap_Q")
    except Exception:
        lvl  = res.level.smoothed if hasattr(res, "level") else 0.0
        trnd = res.trend.smoothed if hasattr(res, "trend") else 0.0
        gap_q = (y - (lvl + trnd)).rename(f"{country}_gap_Q")

    gap_q = gap_q.dropna()
    return res, (gap_q if not gap_q.empty else None)

def _extract_factor_series(res, index):
    fac = res.factors.filtered
    if isinstance(fac, pd.DataFrame):
        s = fac.iloc[:, 0]
        s = (s - s.mean())/s.std() if s.std() else s*0.0
        return pd.Series(s.values, index=index)
    if isinstance(fac, pd.Series):
        s = (fac - fac.mean())/fac.std() if fac.std() else fac*0.0
        return pd.Series(s.values, index=index)

    arr = np.asarray(fac)
    if arr.ndim == 1:
        arr_t = arr
    elif arr.ndim == 2:
        if arr.shape[0] <= arr.shape[1]:
            arr_t = arr[0, :]
        else:
            arr_t = arr[:, 0]
    else:
        nobs = len(index)
        if nobs in arr.shape:
            ax = list(arr.shape).index(nobs)
            arr_t = np.take(arr, indices=0, axis=1-ax) if ax==1 else np.take(arr, indices=0, axis=0)
        else:
            arr_t = arr.ravel()[:len(index)]

    arr_t = np.asarray(arr_t).reshape(-1)
    if arr_t.shape[0] != len(index):
        n = min(len(index), arr_t.shape[0])
        series = pd.Series(np.nan, index=index)
        if n > 0:
            series.iloc[:n] = arr_t[:n]
        s = series
    else:
        s = pd.Series(arr_t, index=index)

    s = (s - s.mean())/s.std() if s.std() else s*0.0
    return s

def _monthly_sample_index(can_std, sample_policy):
    idx = can_std.index
    if sample_policy == "overlap":
        return idx

    cols = []
    for c in ["ip_log", "unemp", "infl_yoy"]:
        if c in can_std.columns and can_std[c].notna().sum() > MIN_OBS:
            cols.append(c)
    if len(cols) == 0:
        return idx

    starts = []
    ends = []
    for c in cols:
        s = can_std[c].dropna()
        if s.empty:
            continue
        starts.append(s.index.min())
        ends.append(s.index.max())
    if len(starts)==0 or len(ends)==0:
        return idx
    start = max(starts); end = min(ends)
    if start >= end:
        return idx
    return pd.date_range(start, end, freq="M")

def fit_mixed_monthly(country, can_std, sample_policy="overlap"):
    idx = _monthly_sample_index(can_std, sample_policy)
    X = pd.DataFrame(index=idx)

    for c in ["ip_log", "unemp", "infl_yoy"]:
        if c in can_std.columns and can_std[c].notna().sum() > MIN_OBS:
            X[c] = can_std[c].reindex(idx)

    if "gdp_log" in can_std.columns and can_std["gdp_log"].notna().sum() > MIN_OBS:
        g_sparse = pd.Series(np.nan, index=idx)
        g_sparse.loc[can_std["gdp_log"].dropna().index.intersection(idx)] = can_std["gdp_log"].dropna().reindex(idx)
        X["gdp_sparse"] = g_sparse

    keep = [c for c in X.columns if X[c].notna().sum() > MIN_OBS]
    X = X[keep]
    if X.shape[1] == 0:
        print(f"[{country}] No usable monthly indicators.")
        return None, None, None

    try:
        mod = DynamicFactor(endog=X, k_factors=1, factor_order=1, error_order=1,
                            enforce_stationarity=False)
        res = mod.fit(disp=False, maxiter=400)
        fac_series = _extract_factor_series(res, index=X.index)
        gap_m = fac_series.rename(f"{country}_gap_M").dropna()
        return res, gap_m, X.index
    except Exception as e:
        print(f"[{country}] DynamicFactor failed: {e}")
        return None, None, X.index

def default_bandwidth(n, exp=0.65):
    m = int(np.clip(n**exp, 10, max(20, n//2)))
    return m

def periodogram(x):
    x = np.asarray(x, float)
    x = x - np.nanmean(x)
    n = len(x)
    fx = np.fft.rfft(x, n=n)[1:]
    I = (1/(2*np.pi*n)) * (fx*np.conj(fx)).real
    w = 2*np.pi*np.arange(1, len(I)+1)/n
    I = np.clip(I, 1e-12, np.inf)
    return w, I

def gph_estimator(x, exp=0.65):
    s = pd.Series(x).dropna()
    n = len(s)
    if n < 40:
        return np.nan, np.nan, {"m": None}
    m = default_bandwidth(n, exp=exp)

    w, I = periodogram(s.values)
    w_m = w[:m]; I_m = I[:m]

    y = np.log(I_m)
    X = np.vstack([np.ones(len(w_m)), -2*np.log(2*np.sin(w_m/2))]).T

    mask = np.isfinite(y) & np.all(np.isfinite(X), axis=1)
    y = y[mask]; X = X[mask, :]
    if len(y) < 10:
        return np.nan, np.nan, {"m": int(m)}

    beta = linalg.lstsq(X, y)[0]
    d_hat = float(beta[1])
    resid = y - X @ beta
    s2 = (resid**2).sum()/max(len(y)-2, 1)
    cov = s2 * np.linalg.inv(X.T @ X)
    se = float(np.sqrt(max(cov[1,1], 0.0)))
    return d_hat, se, {"m": int(m)}

def local_whittle(x, exp=0.65):
    s = pd.Series(x).dropna()
    n = len(s)
    if n < 40:
        return np.nan, np.nan, {"m": None, "success": False}
    m = default_bandwidth(n, exp=exp)

    w = 2*np.pi*np.arange(1, m+1)/n
    fx = np.fft.rfft(s.values, n=n)[1:m+1]
    I = (1/(2*np.pi*n)) * (fx*np.conj(fx)).real
    I = np.clip(I, 1e-12, np.inf)

    lam = 2*np.sin(w/2)
    loglam = np.log(lam)

    def S(d):
        return np.log(np.mean(I * (lam**(2*d)))) - (2*d)*np.mean(loglam)

    opt = optimize.minimize_scalar(S, bounds=(-0.45, 1.45), method='bounded')
    d_hat = float(opt.x)
    se = float(1.0/(2.0*np.sqrt(m)))
    return d_hat, se, {"m": int(m), "success": bool(opt.success), "fun": float(opt.fun)}

def fractional_grid_series(series, label, exps=(0.55, 0.65, 0.80)):
    s = pd.Series(series).dropna()
    rows = []
    for e in exps:
        d_lw, se_lw, ml = local_whittle(s.values, exp=e)
        d_g , se_g , mg = gph_estimator(s.values, exp=e)
        rows.append([e, ml.get("m"), d_lw, se_lw, d_g, se_g, np.nan])
    df = pd.DataFrame(rows, columns=["exp", "m(LW)", "d_LW", "se_LW", "d_GPH", "se_GPH", "d_MLE"])
    return df

def moving_block_boot_diff(gap_Q, gap_M, L=8, B=299, estimator="lw", seed=123):
    x = pd.Series(gap_Q).dropna()
    y = pd.Series(gap_M).dropna()
    common = x.index.intersection(y.index)
    x = x.loc[common]; y = y.loc[common]
    n = len(x)
    if n < 40:
        return None, x, y

    def est(z):
        if estimator == "lw":
            return local_whittle(z.values)[0]
        return gph_estimator(z.values)[0]

    d0, d1 = est(x), est(y)
    d_diff = d1 - d0

    rng = np.random.default_rng(seed)
    blocks = [np.arange(i, min(i+L, n)) for i in range(0, n-L+1)]
    diffs = []
    for _ in range(B):
        idx = []
        while len(idx) < n:
            start = rng.integers(0, len(blocks))
            idx.extend(blocks[start])
        idx = np.asarray(idx[:n], dtype=int)
        xb = x.iloc[idx].reset_index(drop=True)
        yb = y.iloc[idx].reset_index(drop=True)
        diffs.append(est(yb) - est(xb))
    diffs = np.array(diffs)
    p = (np.sum(np.abs(diffs) >= np.abs(d_diff)) + 1)/(B+1)
    return {"d_M_minus_d_Q": float(d_diff), "p_value": float(p), "B":int(B), "L":int(L)}, x, y

def fdr_bh(pvals, alpha=0.05):
    p = np.array(pvals, float)
    n = len(p)
    idx = np.argsort(p)
    thr = alpha * (np.arange(1,n+1)/n)
    passed = p[idx] <= thr
    crit_k = np.where(passed)[0].max()+1 if passed.any() else 0
    cutoff = thr[crit_k-1] if crit_k>0 else 0.0
    return {"cutoff": float(cutoff), "reject_idx_sorted": idx[:crit_k].tolist(), "order": idx.tolist()}

def panel_collect_and_report(summaries, title="Panel"):
    rows = []
    for s in summaries:
        h = s.get("h0_test", None)
        if h is not None and isinstance(h, dict):
            rows.append({"country": s["country"], "dM_minus_dQ": h["d_M_minus_d_Q"], "pval": h["p_value"]})
    if len(rows)==0:
        print(f"\n[{title}] No H0 tests (need both gaps with ≥40 common GDP dates).")
        return None
    df = pd.DataFrame(rows).set_index("country").sort_values("pval")
    print(f"\n[{title}] d_M - d_Q and p-values (sorted):")
    print(df.round(3))
    fdr = fdr_bh(df["pval"].values, alpha=0.05)
    print(f" FDR cutoff ≈ {fdr['cutoff']:.3f}")
    if len(fdr["reject_idx_sorted"])>0:
        print(" Countries rejecting H0 at FDR 5%:",
              ", ".join(df.index[fdr["reject_idx_sorted"]].tolist()))
    else:
        print(" No rejections at FDR 5%.")
    return {"table": df, "fdr": fdr}

def pooled_t_test_diffs(summaries, title="pooled"):
    diffs = []
    for s in summaries:
        h = s.get("h0_test", None)
        if h is not None and isinstance(h, dict):
            diffs.append(h["d_M_minus_d_Q"])
    diffs = np.array(diffs, float)
    diffs = diffs[np.isfinite(diffs)]
    if diffs.size < 3:
        print(f"\n[{title}] insufficient N for pooled t-test.")
        return
    mean = diffs.mean()
    se = diffs.std(ddof=1)/np.sqrt(len(diffs))
    t = mean/se if se>0 else np.nan

    from math import erf, sqrt
    p = 2*(1-0.5*(1+erf(abs(t)/np.sqrt(2)))) if np.isfinite(t) else np.nan
    print(f"\n[{title}] mean(dM-dQ)={mean:.3f}, t={t:.2f}, p={p:.3f}, N={len(diffs)}")

def jensen_liu_demo(n=2000, seed=7):
    rng = np.random.default_rng(seed)
    x = rng.standard_normal(n)
    d_g, _, _ = gph_estimator(x, exp=0.65)
    d_l, _, _ = local_whittle(x, exp=0.65)
    return d_g, d_l

def fractional_summary(series, label, center_exp=0.65):
    s = pd.Series(series).dropna()
    grid = fractional_grid_series(s, label, exps=(0.55, 0.65, 0.80))
    d_lw_c = grid.loc[grid["exp"]==center_exp, "d_LW"].values
    d_c = float(d_lw_c[0]) if len(d_lw_c)>0 else np.nan
    se_c = grid.loc[grid["exp"]==center_exp, "se_LW"].values
    if len(se_c)>0 and np.isfinite(se_c[0]):
        lo = d_c - 1.96*se_c[0]; hi = d_c + 1.96*se_c[0]
    else:
        lo, hi = np.nan, np.nan
    return grid, d_c, (lo, hi)

def temporal_agg_check(gap_M, gap_Q, mf_index, center_exp=0.65):
    if gap_M is None or gap_Q is None:
        return None
    x = pd.Series(gap_M).dropna()
    y = pd.Series(gap_Q).dropna()
    common = x.index.intersection(y.index)
    if len(common) < 40:
        return None
    dm, _, _ = local_whittle(x.loc[common].values, exp=center_exp)
    return dm

def _appendix_tables_from_summaries(summaries, title_prefix):
    rows_bw = []
    for s in summaries:
        c = s["country"]
        grids = s.get("grids", {})
        for freq in ["Q", "M"]:
            g = grids.get(freq, None)
            if isinstance(g, pd.DataFrame) and not g.empty:
                gg = g.copy()
                gg["country"] = c
                gg["freq"] = freq
                rows_bw.append(gg)
    df_bw = None if len(rows_bw)==0 else pd.concat(rows_bw, ignore_index=True)
    if df_bw is not None:
        df_bw = df_bw[["country","freq","exp","m(LW)","d_LW","se_LW","d_GPH","se_GPH","d_MLE"]]
        df_bw = df_bw.sort_values(["country","freq","exp"]).reset_index(drop=True)

    rows_h0 = []
    for s in summaries:
        h = s.get("h0_test", None)
        if isinstance(h, dict):
            rows_h0.append({
                "country": s["country"],
                "dM_minus_dQ": h["d_M_minus_d_Q"],
                "p_value": h["p_value"],
                "B": h.get("B", np.nan),
                "L": h.get("L", np.nan)
            })
    df_h0 = None if len(rows_h0)==0 else pd.DataFrame(rows_h0).sort_values("p_value").reset_index(drop=True)

    if df_bw is not None:
        path_bw = f"/content/appendix_{title_prefix.lower()}_bandwidth.csv"
        df_bw.to_csv(path_bw, index=False)
        print(f"[Saved] {path_bw}")
        print("\n[Appendix – Bandwidth grid]")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 120):
            print(df_bw.round(3))
    if df_h0 is not None:
        path_h0 = f"/content/appendix_{title_prefix.lower()}_h0.csv"
        df_h0.to_csv(path_h0, index=False)
        print(f"[Saved] {path_h0}")
        print("\n[Appendix – H0(dQ=dM) by country]")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 120):
            print(df_h0.round(3))
    return df_bw, df_h0

def process_country(sheet_name, df_raw, sample_policy="overlap", use_diff=False, verbose=True):
    if verbose: print(f"\n=== {sheet_name} ===")
    pairs = parse_sheet_by_position(df_raw)
    quick_data_audit(pairs)

    dfm = build_monthly_panel(pairs)
    quick_panel_audit(dfm)

    can, can_std = transforms(dfm, use_diff=use_diff)

    resQ, gap_Q = fit_uc_quarterly(sheet_name, can)
    if gap_Q is not None and verbose:
        print(f" gap_Q len={len(gap_Q)} {gap_Q.index.min().date()} → {gap_Q.index.max().date()}")

    resM, gap_M, mf_index = fit_mixed_monthly(sheet_name, can_std, sample_policy=sample_policy)
    if gap_M is not None and verbose:
        print(f" [MF sample] {'balanced' if sample_policy=='balanced' else 'union monthly'}")
        print(f" gap_M len={len(gap_M)} {gap_M.index.min().date()} → {gap_M.index.max().date()}")

    center_exp = 0.65
    grid_q = pd.DataFrame(); d_q = np.nan; ci_q = (np.nan, np.nan)
    if gap_Q is not None:
        grid_q, d_q, ci_q = fractional_summary(gap_Q, f"{sheet_name}_Q", center_exp)
        print(f" {sheet_name} Q fractional grid:\n", grid_q.to_string(index=False))
        print(f" {sheet_name} Q d≈{d_q:.3f}  95%CI [{ci_q[0]:.3f}, {ci_q[1]:.3f}] (B={BOOT_B},L={BOOT_L})")
    grid_m = pd.DataFrame(); d_m = np.nan; ci_m = (np.nan, np.nan)
    if gap_M is not None:
        grid_m, d_m, ci_m = fractional_summary(gap_M, f"{sheet_name}_M", center_exp)
        print(f" {sheet_name} M fractional grid:\n", grid_m.to_string(index=False))
        print(f" {sheet_name} M d≈{d_m:.3f}  95%CI [{ci_m[0]:.3f}, {ci_m[1]:.3f}] (B={BOOT_B},L={BOOT_L})")

    if np.isfinite(d_q):
        hl_q = half_life_from_d(d_q)
        print(f" d_Q(center LW exp=0.65) = {d_q:.3f} | Half-life≈{format_halflife_months(hl_q)}")
    if np.isfinite(d_m):
        hl_m = half_life_from_d(d_m)
        print(f" d_M(center LW exp=0.65) = {d_m:.3f} | Half-life≈{format_halflife_months(hl_m)}")

    d_m_on_qdates = None
    if gap_Q is not None and gap_M is not None:
        d_m_on_qdates = temporal_agg_check(gap_M, gap_Q, mf_index, center_exp)
        if d_m_on_qdates is not None and np.isfinite(d_m_on_qdates):
            print(f" [Temporal agg] d(MF→Qdates)={d_m_on_qdates:.3f}, d_Q={d_q:.3f}, diff={(d_m_on_qdates - d_q):.3f}")

    htest = None
    if (gap_Q is not None) and (gap_M is not None):
        htest, x_common, y_common = moving_block_boot_diff(gap_Q, gap_M, L=BOOT_L, B=BOOT_B, estimator="lw", seed=RNG_SEED)
        if htest is not None and verbose:
            print(f" H0(dQ=dM): dM-dQ={htest['d_M_minus_d_Q']:.3f}, p={htest['p_value']:.3f}")

    return {
        "country": sheet_name,
        "frac": {"Q": (d_q, ci_q), "M": (d_m, ci_m), "M_on_Q": d_m_on_qdates},
        "grids": {"Q": grid_q, "M": grid_m},
        "h0_test": htest
    }

def run_pipeline(book, sample_policy="overlap", use_diff=False, title="RUN"):
    print("\n" + "="*72)
    print(f"{title.upper():<}  |  sample_policy={sample_policy}  use_diff={use_diff}")
    print("="*72)
    summaries = []
    for sh, df in book.items():
        try:
            res = process_country(sh, df, sample_policy=sample_policy, use_diff=use_diff, verbose=True)
            summaries.append(res)
        except Exception as e:
            print(f"[{sh}] ERROR: {e}")
    panel = panel_collect_and_report(summaries, title=f"{title} Panel")
    pooled_t_test_diffs(summaries, title=f"{title} pooled")

    _appendix_tables_from_summaries(summaries, title_prefix=title.replace(" ", "_").lower())
    return summaries, panel

def main():
    print("Loading workbook...")

    d_g, d_l = jensen_liu_demo(n=2000, seed=7)
    print(f"\n[Jensen–Liu demo] d_gph≈{d_g:.3f}, d_lw≈{d_l:.3f} (true d=0)")

    xl = pd.ExcelFile(DATA_PATH)
    names = G7_SHEETS if G7_SHEETS is not None else xl.sheet_names
    book = {sh: pd.read_excel(xl, sheet_name=sh) for sh in names}

    _ = run_pipeline(book,
                     sample_policy=BASELINE_POLICY["sample_policy"],
                     use_diff=BASELINE_POLICY["use_diff"],
                     title="Baseline Run")

    _ = run_pipeline(book,
                     sample_policy=ROBUSTNESS_POLICY["sample_policy"],
                     use_diff=ROBUSTNESS_POLICY["use_diff"],
                     title="Robustness Run")

if __name__ == "__main__":
    main()


Loading workbook...

[Jensen–Liu demo] d_gph≈0.050, d_lw≈0.054 (true d=0)

BASELINE RUN  |  sample_policy=overlap  use_diff=False

=== Canada ===
---- DATA AUDIT (raw pairs) ----
   gdp: n=257, start=1961-01-31, end=2025-01-31
   cpi: n=65, start=1960-01-31, end=2024-01-31
  rate: n=768, start=1960-01-31, end=2023-12-31
 unemp: n=848, start=1955-01-31, end=2025-08-31
    ip: n=758, start=1961-01-31, end=2024-02-29
---- PANEL AUDIT (union monthly) ----
       gdp: n=257, start=1961-01-31, end=2025-01-31
       cpi: n=65, start=1960-01-31, end=2024-01-31
      rate: n=768, start=1960-01-31, end=2023-12-31
     unemp: n=848, start=1955-01-31, end=2025-08-31
        ip: n=758, start=1961-01-31, end=2024-02-29
 gap_Q len=257 1961-01-31 → 2025-01-31
 [MF sample] union monthly
 gap_M len=848 1955-01-31 → 2025-08-31
 Canada Q fractional grid:
  exp  m(LW)     d_LW    se_LW    d_GPH   se_GPH  d_MLE
0.55     21 0.814664 0.109109 0.881667 0.072857    NaN
0.65     36 0.998738 0.083333 1.066701 0.0